# Using BeRTA with Python
In this tutorial, you'll learn how to control BeRTA (Binaural Rendering Toolbox Application) using Python to generate spatial audio.

By the end of this notebook, you will be able to:
- Launch and connect to BeRTA using Open Sound Control (OSC)
- Download and load Head-Related Transfer Functions (HRTFs)
- Control audio sources in 3D space
- Present and record dynamic spatial audio scenes


## Prerequisites

### Required Software
- **BeRTA Renderer** installed on your system
- **Python 3.12** with the following packages (other version of Python have not been tested here):
- **The following python packages:**

```bash
pip install python-osc wget spatialaudiometrics
```

### Required Files
- Two audio files (e.g., `stimuli/input/guitar_ebusqam.wav`)
- You can use any `.wav` file - just update the path accordingly

---

## Part 1: Imports and Setup

Let's start by importing all the libraries we'll need:


In [1]:
import subprocess
from pythonosc import udp_client
from pythonosc.dispatcher import Dispatcher
from pythonosc.osc_server import BlockingOSCUDPServer
import time
import wget
import os 
from spatialaudiometrics import angular_metrics as am

**What are these libraries for?**
- `subprocess` - Launch external applications (BeRTA)
- `pythonosc` - Send/receive OSC messages
- `wget` - Download HRTF files from the internet
- `spatialaudiometrics` - Convert between coordinate systems
- `sounddevice` - Play audio outside of BRT

---

## Part 2: Launching BeRTA

In [2]:
# Update this path to match your BeRTA installation
berta_path = "C:/Program Files/University of Malaga/BeRTA-Renderer App/BeRTA-Renderer.exe"

# Launch BeRTA as a subprocess
berta_process = subprocess.Popen(berta_path)

# If you wanted to load in your own settings file
# settings_path = "example_settings.json"
# berta_process = subprocess.Popen(berta_path + ' ' + settings_path)

# Wait for the application to initialise 
time.sleep(8)
print('BeRTA loaded')

BeRTA loaded



⏳ **Why use time.sleep()?** BeRTA needs time to fully start up before it can receive OSC messages. If you send messages too early, they'll be ignored.

Once loaded, open the settings to select your preferred audio interface to present the audio

---

## Part 3: Establishing OSC Communication

Open Sound Control (OSC) is a network protocol for communication between multimedia devices. We need to set up both a **client** (to send messages) and a **server** (to receive responses).

In [3]:
# Configure connection parameters
ip = "127.0.0.1"
send_port = 10017
receive_port = 8000

# Create OSC client to SEND messages TO BeRTA
berta_client = udp_client.SimpleUDPClient(ip, send_port)

# Create OSC server to RECEIVE messages FROM BeRTA
dispatcher = Dispatcher()

# You can  use a catch-all handler for debugging
dispatcher.map("/*", lambda addr, *args: print(f"DEBUG - Received {addr}: {args}"))

server = BlockingOSCUDPServer((ip, receive_port), dispatcher)

print('Connected...')

Connected...


**Key Concepts:**
- `127.0.0.1` = localhost (your own computer). This can be changed to the IP of another computer if you would like to run BeRTA on one machine and the Python code on another
- Port `10017` = BeRTA's default listening port
- Port `8000` = Python's listening port (for receiving responses)

---

## Part 4: Testing the Connection

Let's establish a two-way connection and verify it works:

In [4]:
# Tell BeRTA to connect back to Python on port 8000
berta_client.send_message("/control/connect", [ip, receive_port])

# Give BeRTA a moment to establish the connection
time.sleep(0.5)

# Handle the connection confirmation (BeRTA echoes back the connect message)
print("Waiting for connection confirmation...")
server.handle_request()
print("Connection established!\n")

# Send a ping message
print("Sending ping to BeRTA...")
berta_client.send_message("/control/ping", "Hello from Python!")

# Wait for and handle the pong response
print("Waiting for pong response...")
server.handle_request()  # This blocks until we receive the /control/pong message

Waiting for connection confirmation...
DEBUG - Received /control/connect: ('localhost', 10017)
Connection established!

Sending ping to BeRTA...
Waiting for pong response...
DEBUG - Received /control/ping: ()


**💡 Note:** Not all OSC commands in BeRTA return responses! Check BeRTA's documentation to see which commands send data back vs. which only display in the console.

---

## Part 5: Resetting BeRTA

BeRTA loads with default settings. Let's clear everything and start fresh using OSC messages:

In [5]:
# Remove all existing audio sources
berta_client.send_message("/removeAllSources", [])

# Disable reverb model
berta_client.send_message("/enableModel", ["ReverbPath", 0])

# Disable distance attenuation (free field model)
berta_client.send_message("/enableModel", ["FreeField", 0])

## Part 6: Loading an HRTF

Head-Related Transfer Functions (HRTFs) are crucial for realistic 3D audio. They capture how sound is filtered by your head, ears, and torso from different directions.

We'll download an HRTF from the **SONICOM database**, a collection of acoustically measured HRTFs at Imperial, and load it into BeRTA


In [6]:
# Select a participant from the SONICOM database
p_no = 'P0107'

# Construct download URL
link = f"ftp://transfer.ic.ac.uk:2122/2022_SONICOM-HRTF-DATASET/{p_no}/HRTF/HRTF/48kHz/{p_no}_FreeFieldComp_NoITD_48kHz.sofa"
outfilename = f"{p_no}_hrtf.sofa"

# Download the HRTF file
if not os.path.exists(outfilename):
    print(f"Downloading HRTF for participant {p_no}...")
    wget.download(link, outfilename)
    print("\nDownload complete!")

sofa_path = os.getcwd() + "/" + outfilename

# Load the HRTF into BeRTA
# Parameters: [identifier, file_path, interpolation_step]
berta_client.send_message("/resources/loadHRTF", [p_no, sofa_path, 5.0])
print('\nLoaded HRTF')


Loaded HRTF


---

## Part 7: Loading and Positioning an Audio Source

Now let's load an audio file and place it in 3D space:

In [7]:
# Path to your audio file
source_path = os.getcwd() + "/stimuli/input/guitar_ebusqam.wav"

# Load the audio source
berta_client.send_message("/source/loadSource", ['guitar', source_path, 'DirectivityModel'])

# Define position in spherical coordinates (more intuitive!)
az = 0        # Azimuth: 0° = front, 90° = left, 180° = back, 270° = right
el = 0        # Elevation: 0° = ear level, 90° = above, -90° = below
dist = 1.5    # Distance in meters

# Convert to Cartesian coordinates (x, y, z)
[x, y, z] = am.polar2cartesian(az, el, dist)

# Set the source location
berta_client.send_message("/source/location", ["guitar", x, y, z])
print('\nLoaded source')


Loaded source


**Coordinate Systems:**
- **Spherical** (azimuth, elevation, distance) - Spatial-audio friendly. Spatial Audio Metrics can also convert between spherical and polar coordinate systems
- **Cartesian** (x, y, z) - What BeRTA uses internally

The `spatialaudiometrics` library handles the conversion

---
## Part 8: Playing and Animating the Source

Let's play the audio and move it around the listener!

In [8]:
# Start playback
berta_client.send_message("/source/play", "guitar")

# Animate the source in a circle (5 complete rotations)
print("Moving source around the listener...")
while az < 360 * 2:
    # Convert current azimuth to Cartesian coordinates
    [x, y, z] = am.polar2cartesian(az, el, dist)
    
    # Update source position
    berta_client.send_message("/source/location", ["guitar", x, y, z])
    
    # Increment azimuth and wait
    az += 5
    time.sleep(0.2)

# Stop playback
berta_client.send_message("/source/stop", "guitar")

Moving source around the listener...


---
🎧 **Put on your headphones!** You should hear the guitar circling around you 2 times

---
## Part 9: Adding sources and recording
Now lets add another source and record the scene. Since we want to move the sources and record we shall use the /playAndRecord OSC command. The /record OSC command records offline and therefore cannot let you move the sources.

**💡 Note:** Currently only recording in .mat is supported with .wav being introduced later.


In [9]:
# Lets load in another source and initialise the positions
source_path = os.getcwd() + "/stimuli/input/speech_1_ebusqam.wav"
berta_client.send_message("/source/loadSource",['speech',source_path,'DirectivityModel'])
az2 = 90
el2 = 20
dist = 1.5 # in meters
[x,y,z] = am.polar2cartesian(az2, el2, dist)
berta_client.send_message("/source/location",["speech",x,y,z])

az = 0
el = 0
[x,y,z] = am.polar2cartesian(az, el, dist)
berta_client.send_message("/source/location",["guitar",x,y,z])

# Play and record as a .mat for 30 seconds
recording_duration = 30  # seconds
berta_client.send_message("/playAndRecord",["C:/GitHubRepos/BRT_Spatial_Attention_Experiment/test.mat","mat",recording_duration]) 

# Record start time
start_time = time.time()

# Then lets change the direction of the sound
while (time.time() - start_time) < recording_duration:
    [x,y,z] = am.polar2cartesian(az, el, dist)
    berta_client.send_message("/source/location",["guitar",x,y,z])
    
    # Check if we're halfway through
    elapsed_time = time.time() - start_time
    if (elapsed_time >= recording_duration / 2) and (elapsed_time < (recording_duration / 2 + 0.1)):
        # And lets add a bit of reverb part way
        berta_client.send_message("/enableModel",["ReverbPath",1])
    
    az += 5
    el2 += 1
    az2 += -3
    [x2,y2,z2] = am.polar2cartesian(az2,el2,dist)
    berta_client.send_message("/source/location",["speech",x2,y2,z2])
    time.sleep(0.1)

berta_client.send_message("/stop",[])

## Part 10: Cleanup (Optional)

When you're done experimenting, you can close BeRTA:

In [10]:
# This will terminate the BeRTA process
berta_process.terminate()

--- 
## Experiments to Try

Now that you understand the basics, try modifying the code:

1. **Change the starting position** - Start the source behind you (`az = 180`)
2. **Adjust rotation speed** - Make it faster (`time.sleep(0.1)`) or slower
3. **Change distance** - Try `dist = 0.5` for a more closer source and try enabling and disabling the near-field model
4. **Add elevation changes** - Make the source spiral up and down
5. **Try different HRTFs** - Change `p_no` to 'P0108' or another participant from the SONICOM dataset
6. **Variable speed** - Make the source slow down and speed up
7. **Load multiple sources** - Add more sources at different positions
8. **Restrict source to certain quadrants** - Create more complex, random and restricted movement paths
9. **Interactive control** - Add keyboard controls for real-time positioning
--- 

## Resources

- **BeRTA App:** https://github.com/GrupoDiana/BRTLibrary
- **BeRTA Documentation:** https://grupodiana.github.io/BRT-Documentation/osc/
- **SONICOM HRTF Database:** https://www.axdesign.co.uk/tools-and-devices/sonicom-hrtf-dataset
- **Spatial Audio Metrics:** https://github.com/Katarina-Poole/Spatial-Audio-Metrics

---